<a href="https://colab.research.google.com/github/koki-1231/Chemistry-experiment/blob/main/lecture04_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第4回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール


- 訓練データは`x_train`， `t_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train`，`t_train`以外の学習データは使わないでください．**
- Pytorchを利用して構いません．
- ただし，**`torch.nn.Conv2d`のような高レベルのAPIは使用しないで下さい**．具体的には，`nn.Parameter`, `nn.Module`, `nn.Sequential`以外の`nn`系のAPIです．使用した場合エラーになります．
- torchvision等で既に実装されているモデルも使用しないで下さい．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルをcsv形式で保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化」を選択して**提出してください．
    2. それに対応するpythonのコードを　ファイル＞ダウンロード＞.pyをダウンロード　から保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコード全体をコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`t_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 作業ディレクトリを指定
work_dir = 'drive/MyDrive/Colab Notebooks/DLBasics2025_colab'

### データの読み込み（この部分は修正しないでください）

`__len__`は，Pythonの組み込み関数len()を呼んだときに，内部で呼ばれる特殊メソッドです．

`__getitem__`は，インデックスやキーで要素を取得するときに，内部で呼ばれる特殊メソッドです．

どちらも， Datasetクラスを自作する際によく登場します．

```python
class MyList:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

mylist = MyList([10, 20, 30])
print(len(mylist))  # __len__が呼び出される
# 3
print(mylist[1])  # __getitem__が呼び出される
# 20
```

In [3]:
# API制限のコードと，torchに統合されているdynamoライブラリの挙動がコンフリクトを起こすため，dynamoを無効化
import torch._dynamo
torch._dynamo.disable()

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect

# 使用できるAPIを制限
nn_except = ["Module", "Parameter", "Sequential"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0])

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 学習データ
x_train = np.load(work_dir + '/Lecture04/data/x_train.npy')
t_train = np.load(work_dir + '/Lecture04/data/y_train.npy')

# テストデータ
x_test = np.load(work_dir + '/Lecture04/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 多層パーセプトロンの実装

In [ ]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
def relu(x):
    # WRITE ME


def softmax(x):
    # WRITE ME


class Dense(nn.Module):  # nn.Moduleを継承する
    # WRITE ME


class MLP(nn.Module):  # nn.Moduleを継承する
    # WRITE ME

in_dim = 784
hid_dim = 200
out_dim = 10
lr = 0.001
n_epochs = 10


mlp = # WRITE ME

optimizer = # WRITE ME

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        # WRITE ME

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    mlp.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        # WRITE ME

        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        train_true_num/train_num,
        np.mean(losses_valid),
        valid_true_num/valid_num
    ))

In [ ]:
mlp.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = mlp.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv(work_dir + '/Lecture04/submission_pred_04.csv', header=True, index_label='id')

In [1]:
import os
import inspect
import copy
import math
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ===================================================================
# 0. 環境設定とAPI制限 (★改善点 1: 「コード1」の安全なAPI制限を導入)
# ===================================================================

# --- Google Driveマウントと作業ディレクトリ設定 ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    # ★★★ ご自身の環境に合わせて修正してください ★★★
    work_dir = '/content/drive/MyDrive/Colab Notebooks/DLBasics2025_colab'
    if not os.path.exists(work_dir):
        print(f"警告: ディレクトリ '{work_dir}' が存在しません。'.' を使用します。")
        work_dir = '.'
except ImportError:
    print("Google Colab 環境ではありません。カレントディレクトリ '.' を使用します。")
    work_dir = '.'
print(f"作業ディレクトリ: {work_dir}")


# ★★★ 改善点 1: 「コード1」で使われていた安全なAPI制限に変更 ★★★
# optim.Adamが依存する nn.functional や nn.attention などの
# 内部モジュールを削除しないようにします。
nn_except = ["Module", "Parameter", "Sequential", "parameter", "init", "CrossEntropyLoss", "ModuleList"] # Added ModuleList

for m in inspect.getmembers(nn):
    attr_name = m[0]
    attr_value = m[1]

    # 除外リストまたはプライベート属性はスキップ
    if attr_name in nn_except or attr_name.startswith("__"):
        continue

    # ★重要: モジュール型の属性は削除しない（attention, functional等）
    if inspect.ismodule(attr_value):
        continue

    # それ以外のクラス・関数(nn.Linear, nn.ReLUなど)を削除
    try:
        delattr(nn, attr_name)
    except (AttributeError, TypeError):
        pass

print(f"--- API制限適用完了 (許可: {nn_except} + サブモジュール) ---")
# ★★★ 改善完了 ★★★


# --- シード固定関数 ---
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # アンサンブルのためにallも設定
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# --- デバイス設定 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")


# ===================================================================
# 1. データ準備 (★改善点 2: Cutoutのバグ修正)
# ===================================================================

def augment_data_torch_cutout(images_flat, h_flip_prob=0.5, shift_pixels=2, cutout_size=8):
    """
    torchvisionに依存しない、テンソル演算のみによるデータ拡張。
    """
    images_2d = images_flat.reshape(-1, 1, 28, 28)
    n_images, c, h, w = images_2d.shape

    # --- 1. 水平フリップ (p=0.5) ---
    flip_mask = torch.rand(n_images, device=images_flat.device) < h_flip_prob
    if flip_mask.any():
        images_2d[flip_mask] = torch.flip(images_2d[flip_mask], dims=[3])

    # --- 2. ランダムシフト (±2ピクセル) ---
    shifts = torch.randint(-shift_pixels, shift_pixels + 1, (n_images, 2), device=images_flat.device)
    # nn.functional.pad は安全なAPI制限により削除されていないため使用可能
    padded_images = torch.nn.functional.pad(images_2d, (shift_pixels, shift_pixels, shift_pixels, shift_pixels), mode='constant', value=0.0)
    augmented_shifted = torch.empty_like(images_2d)

    for i in range(n_images):
        shift_y, shift_x = shifts[i, 0].item(), shifts[i, 1].item()
        y_start = shift_pixels + shift_y
        x_start = shift_pixels + shift_x
        augmented_shifted[i] = padded_images[i, :, y_start:y_start+h, x_start:x_start+w]

    images_2d = augmented_shifted

    # --- 3. カットアウト (Cutout) ---
    cutout_half = cutout_size // 2
    center_y = torch.randint(0, h, (n_images,), device=images_flat.device)
    center_x = torch.randint(0, w, (n_images,), device=images_flat.device)

    for i in range(n_images):
        y1 = torch.clamp(center_y[i] - cutout_half, 0, h).item()
        y2 = torch.clamp(center_y[i] + cutout_half, 0, h).item()

        # ★★★ 改善点 2: バグ修正 ★★★
        # x1 と x2 の計算が両方 '+' になっていたのを修正
        # x1 = torch.clamp(center_x[i] + cutout_half, 0, w).item() # <- バグ
        # x2 = torch.clamp(center_x[i] + cutout_half, 0, w).item() # <- バグ
        x1 = torch.clamp(center_x[i] - cutout_half, 0, w).item() # <- 修正
        x2 = torch.clamp(center_x[i] + cutout_half, 0, w).item() # <- 修正

        # 範囲チェック (y1 < y2 and x1 < x2)
        if y1 < y2 and x1 < x2:
            images_2d[i, :, y1:y2, x1:x2] = 0.0
    # ★★★ 改善完了 ★★★

    return images_2d.reshape(-1, 784)

# --- データセットクラス (torchvision 依存の排除) ---
# 元の .ipynb にあったシンプルなDatasetクラス
class FashionDataset(Dataset):
    def __init__(self, x_data, t_label):
        # ★改善: 0-1に正規化 (augment_data_torch_cutoutは0-1を前提)
        self.x_data = x_data.reshape(-1, 784).astype(np.float32) / 255.0
        self.t_label = t_label
        self.t_hot = np.eye(10)[t_label].astype(np.float32)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        # テンソルに変換して返す
        img_tensor = torch.tensor(self.x_data[idx], dtype=torch.float32)
        label = torch.tensor(self.t_label[idx], dtype=torch.long)
        hot_label = torch.tensor(self.t_hot[idx], dtype=torch.float32)

        # (Tensor[784], Tensor[10], Tensor[long]) を返す
        return img_tensor, hot_label, label

class TestDataset(Dataset):
    def __init__(self, x_data):
        self.x_data = x_data.reshape(-1, 784).astype(np.float32) / 255.0

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        img_tensor = torch.tensor(self.x_data[idx], dtype=torch.float32)
        return img_tensor # テスト時は画像データのみ返す

# --- データのロードと分割 ---
data_path = os.path.join(work_dir, 'Lecture04', 'data')
x_train_all = np.load(os.path.join(data_path, 'x_train.npy'))
t_train_all = np.load(os.path.join(data_path, 'y_train.npy'))
x_test = np.load(os.path.join(data_path, 'x_test.npy'))

x_train_split, x_valid_split, t_train_split_label, t_valid_split_label = train_test_split(
    x_train_all, t_train_all, test_size=0.2, random_state=42
)
print(f"訓練データ: {x_train_split.shape}, 検証データ: {x_valid_split.shape}")

# --- データローダーの作成 ---
batch_size = 128
num_workers = 2

train_data = FashionDataset(x_train_split, t_train_split_label)
valid_data = FashionDataset(x_valid_split, t_valid_split_label)
test_data = TestDataset(x_test)

dataloader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
dataloader_valid = DataLoader(valid_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
dataloader_test = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

print("--- データローダー準備完了 (torchvision非依存) ---")


# ===================================================================
# 2. PyTorchによるニューラルネットワークのクラス化 (ルール準拠)
# ===================================================================

def relu(x):
    return torch.where(x > 0, x, torch.zeros_like(x))

class Dense(nn.Module):
    def __init__(self, in_dim, out_dim, is_relu=True):
        super().__init__()
        self.W = nn.Parameter(torch.empty(in_dim, out_dim, dtype=torch.float32))
        self.b = nn.Parameter(torch.empty(out_dim, dtype=torch.float32))
        self.initialize_parameters(is_relu)

    def initialize_parameters(self, is_relu):
        if is_relu:
            nn.init.kaiming_normal_(self.W, mode='fan_in', nonlinearity='relu')
        else:
            nn.init.xavier_normal_(self.W)
        nn.init.zeros_(self.b)

    def forward(self, x):
        return torch.matmul(x, self.W) + self.b

class BatchNorm1d(nn.Module):
    def __init__(self, dim, momentum=0.9, eps=1e-8):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(dim).float())
        self.beta = nn.Parameter(torch.zeros(dim).float())
        self.register_buffer('running_mean', torch.zeros(dim).float())
        self.register_buffer('running_var', torch.ones(dim).float())
        self.momentum = momentum
        self.eps = eps

    def forward(self, x):
        if self.training:
            if x.shape[0] > 1:
                mu = x.mean(axis=0)
                var = x.var(axis=0, unbiased=False)
            else:
                mu = x.mean(axis=0)
                var = torch.zeros_like(mu)
            self.running_mean = self.momentum * self.running_mean.detach() + (1.0 - self.momentum) * mu.detach()
            self.running_var = self.momentum * self.running_var.detach() + (1.0 - self.momentum) * var.detach()
            x_norm = (x - mu) / torch.sqrt(var + self.eps)
        else:
            x_norm = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)
        return self.gamma * x_norm + self.beta

class Dropout(nn.Module):
    def __init__(self, dropout_ratio=0.5):
        super().__init__()
        self.dropout_ratio = dropout_ratio

    def forward(self, x):
        if self.training and self.dropout_ratio > 0.0:
            drop_mask = (torch.rand_like(x) > self.dropout_ratio).float() / (1.0 - self.dropout_ratio)
            return x * drop_mask
        else:
            return x

class MLP(nn.Module):
    def __init__(self, in_dim, hidden_sizes, out_dim, dropout_ratio, bn_momentum):
        super().__init__()
        self.fc1 = Dense(in_dim, hidden_sizes[0], is_relu=True)
        self.bn1 = BatchNorm1d(hidden_sizes[0], momentum=bn_momentum)
        self.drop1 = Dropout(dropout_ratio)
        self.fc2 = Dense(hidden_sizes[0], hidden_sizes[1], is_relu=True)
        self.bn2 = BatchNorm1d(hidden_sizes[1], momentum=bn_momentum)
        self.drop2 = Dropout(dropout_ratio)
        self.fc3 = Dense(hidden_sizes[1], hidden_sizes[2], is_relu=True)
        self.bn3 = BatchNorm1d(hidden_sizes[2], momentum=bn_momentum)
        self.drop3 = Dropout(dropout_ratio)
        self.output_layer = Dense(hidden_sizes[2], out_dim, is_relu=False)

    def forward(self, x):
        # 入力は (B, 784)
        x = relu(self.drop1(self.bn1(self.fc1(x))))
        x = relu(self.drop2(self.bn2(self.fc2(x))))
        x = relu(self.drop3(self.bn3(self.fc3(x))))
        x = self.output_layer(x)
        return x # ロジットを返す


# ===================================================================
# 3. アンサンブル学習ループ (★改善点 2, 3, 4)
# ===================================================================

# --- ハイパーパラメータ設定 ---
in_dim = 784
out_dim = 10
hidden_sizes = [1024, 512, 256]
bn_momentum = 0.9
lr = 0.001
reg_lambda = 1e-5

# ★アイデア 2: 過学習抑制のためDropoutを強化
dropout_ratio = 0.4 # 0.3 -> 0.4 (0.5なども試す価値あり)

# ★要望: Epoch 500
n_epochs = 500
patience = 60 # 60エポック (約600イテレーション) 改善がなければ停止

# ★要望: アンサンブル
ensemble_seeds = [42, 123, 555] # 3つの異なるシードでモデルを訓練
all_best_model_states = []
all_best_valid_accs = []

criterion = nn.CrossEntropyLoss()

print(f"--- アンサンブル学習開始 (Models: {len(ensemble_seeds)}, Epochs: {n_epochs}, Dropout: {dropout_ratio}) ---")
print("--- ★データ拡張: 手動Cutout + Shift + Flip (torchvision非依存) ---")

for i, seed in enumerate(ensemble_seeds):
    print(f"\n--- モデル {i+1}/{len(ensemble_seeds)} (Seed: {seed}) の学習開始 ---")

    # ループごとにシードを固定
    set_seed(seed)

    # モデル、オプティマイザ、スケジューラを初期化
    mlp = MLP(in_dim, hidden_sizes, out_dim, dropout_ratio, bn_momentum).to(device)
    optimizer = optim.Adam(mlp.parameters(), lr=lr, weight_decay=reg_lambda)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

    best_valid_accuracy = 0.0
    patience_counter = 0
    best_model_state = None

    for epoch in range(n_epochs):
        losses_train = []
        train_num = 0
        train_true_num = 0

        mlp.train()  # 訓練時には勾配を計算するtrainモードにする
        # dataloader_train は (img_tensor[784], t_hot[10], t_label[long]) を返す
        for x, t_hot, t_label in dataloader_train:
            x, t_label = x.to(device), t_label.to(device)

            # --- ★手動データ拡張 (Cutout + Shift + Flip) ---
            x_aug = augment_data_torch_cutout(x,
                                                h_flip_prob=0.5,
                                                shift_pixels=2,
                                                cutout_size=8)

            y_logits = mlp(x_aug) # 拡張後のデータで学習
            loss = criterion(y_logits, t_label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses_train.append(loss.tolist())
            pred = y_logits.argmax(1)
            acc = (pred == t_label).float()
            train_num += acc.size(0)
            train_true_num += acc.sum().item()

        # --- 検証 (★手動TTA: 水平フリップ) ---
        mlp.eval()  # 評価時には勾配を計算しないevalモードにする
        losses_valid = []
        valid_num = 0
        valid_true_num = 0
        with torch.no_grad():
            for x, t_hot, t_label in dataloader_valid:
                x, t_label = x.to(device), t_label.to(device)

                # TTA (1): 元の画像 (B, 784)
                y_logits_original = mlp(x)

                # TTA (2): 水平反転した画像
                x_reshaped = x.reshape(-1, 1, 28, 28)
                x_flipped = torch.flip(x_reshaped, dims=[3]) # (B, 1, 28, 28)
                x_flipped_flat = x_flipped.reshape(-1, 784) # (B, 784)
                y_logits_flipped = mlp(x_flipped_flat)

                y_logits_avg = (y_logits_original + y_logits_flipped) / 2.0

                loss = criterion(y_logits_avg, t_label)
                losses_valid.append(loss.tolist())

                pred = y_logits_avg.argmax(1)
                acc = (pred == t_label).float()
                valid_num += acc.size(0)
                valid_true_num += acc.sum().item()

        scheduler.step()

        current_train_accuracy = train_true_num / train_num
        current_valid_accuracy = valid_true_num / valid_num
        current_lr = optimizer.param_groups[0]['lr']

        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'EPOCH: {epoch+1:3d}, Train [Loss: {np.mean(losses_train):.4f}, Acc: {current_train_accuracy:.4f}], '
                  f'Valid [Loss: {np.mean(losses_valid):.4f}, Acc(TTA): {current_valid_accuracy:.4f}], '
                  f'LR: {current_lr:.6f}')

        if current_valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = current_valid_accuracy
            patience_counter = 0
            best_model_state = copy.deepcopy(mlp.state_dict())
            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f"    -> Best TTA Valid Accuracy Updated: {best_valid_accuracy:.4f}")
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"\n--- Early stopping triggered after {epoch+1} epochs. ---")
            break

    # このシードでのベストモデルと精度を保存
    all_best_model_states.append(best_model_state)
    all_best_valid_accs.append(best_valid_accuracy)
    print(f"--- モデル {i+1} 完了 (Best Valid Acc: {best_valid_accuracy:.4f}) ---")


# ===================================================================
# 4. 最終予測 (アンサンブルと手動TTA)
# ===================================================================
print("\n--- 全モデルの学習完了 ---")
print(f"各モデルのベスト検証精度 (TTA): {[round(acc, 4) for acc in all_best_valid_accs]}")
print(f"平均検証精度 (TTA): {np.mean(all_best_valid_accs):.4f}")
print("\n--- アンサンブルとTTAによる最終予測の生成 ---")

all_test_logits = [] # [モデル数] x [データ数, クラス数] のリスト

# MLPモデルのインスタンスを再作成 (重みをロードするため)
mlp = MLP(in_dim, hidden_sizes, out_dim, dropout_ratio, bn_momentum).to(device)
mlp.eval() # 常に評価モード

for model_state in all_best_model_states:
    if model_state is None:
        print("警告: 訓練されなかったモデル（stateがNone）をスキップします。")
        continue

    mlp.load_state_dict(model_state)

    model_test_logits = [] # このモデルの予測（ロジット）

    with torch.no_grad():
        for x in dataloader_test:
            x = x.to(device)

            # TTA (1): 元の画像
            y_logits_original = mlp.forward(x)

            # TTA (2): 水平反転した画像
            x_reshaped = x.reshape(-1, 1, 28, 28)
            x_flipped = torch.flip(x_reshaped, dims=[3])
            x_flipped_flat = x_flipped.reshape(-1, 784)
            y_logits_flipped = mlp.forward(x_flipped_flat)

            y_logits_avg = (y_logits_original + y_logits_flipped) / 2.0

            model_test_logits.append(y_logits_avg.cpu()) # CPUに集める

    # このモデルの全テストデータに対する予測を結合
    all_test_logits.append(torch.cat(model_test_logits, dim=0))

if not all_test_logits:
    print("エラー: 有効なモデルが1つも訓練されませんでした。")
else:
    # (モデル数, データ数, クラス数) のテンソルにスタック
    stacked_logits = torch.stack(all_test_logits, dim=0)

    # (モデル数) の軸で平均を取る (アンサンブル)
    mean_logits = torch.mean(stacked_logits, dim=0) # (データ数, クラス数)

    # 最終予測
    final_pred = mean_logits.argmax(1).tolist()

    # --- 提出ファイルの保存 ---
    submission = pd.Series(final_pred, name='label')
    submission_filename = os.path.join(work_dir, 'Lecture04', 'submission_pred_06_ensemble_tta.csv')
    os.makedirs(os.path.dirname(submission_filename), exist_ok=True)
    submission.to_csv(submission_filename, header=True, index_label='id')
    print(f"\n--- 予測完了。提出ファイル '{submission_filename}' を保存しました。 ---")

Mounted at /content/drive
作業ディレクトリ: /content/drive/MyDrive/Colab Notebooks/DLBasics2025_colab
--- API制限適用完了 (許可: ['Module', 'Parameter', 'Sequential', 'parameter', 'init', 'CrossEntropyLoss', 'ModuleList'] + サブモジュール) ---
使用デバイス: cuda
訓練データ: (48000, 28, 28), 検証データ: (12000, 28, 28)
--- データローダー準備完了 (torchvision非依存) ---
--- アンサンブル学習開始 (Models: 3, Epochs: 500, Dropout: 0.4) ---
--- ★データ拡張: 手動Cutout + Shift + Flip (torchvision非依存) ---

--- モデル 1/3 (Seed: 42) の学習開始 ---


AttributeError: module 'torch.nn' has no attribute 'ModuleDict'

アンサンブルの改善

In [1]:
import math
import copy
import time
import random
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# API制限のコードと，torchに統合されているdynamoライブラリの挙動がコンフリクトを起こすため，dynamoを無効化
import torch._dynamo
torch._dynamo.disable()

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import inspect


# === API制限 (ユーザー提供コード) ===
# ModuleListと parameter を追加
nn_except = ["Module", "Parameter", "Sequential", "ModuleList", "parameter"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0])
# --- API制限ここまで ---


# ===================================================================
# 0. グローバル設定 (シード固定, デバイス)
# ===================================================================
def set_seed(seed):
    """シードを固定して再現性を確保する"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

# データ分割用のシード (Numpy版と合わせる)
BASE_SEED = 42
set_seed(BASE_SEED) # まずはベースシードで固定

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===================================================================
# 1. データ読み込みと前処理
# ===================================================================
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True) # 再マウントを強制
    # (!!! この部分はご自身の環境に合わせて必ず設定してください !!!)
    work_dir = '/content/drive/MyDrive/Colab Notebooks/DLBasics2025_colab' # ← ★★★ 要変更 ★★★

    if not os.path.exists(work_dir):
        print(f"警告: ディレクトリ '{work_dir}' が存在しません。")
        work_dir = '.'
        data_dir = work_dir + '/Lecture04/data'
    else:
        data_dir = work_dir + '/Lecture04/data'

except ImportError:
    print("Google Colab 環境ではないため、ローカルカレントディレクトリを参照します。")
    work_dir = '.'
    data_dir = work_dir + '/Lecture04/data' # 宿題のパスを想定

print(f"データディレクトリ: {data_dir}")
print(f"デバイス: {device}")

# 学習データ
try:
    x_train = np.load(data_dir + '/x_train.npy')
    t_train = np.load(data_dir + '/y_train.npy')
    # テストデータ
    x_test = np.load(data_dir + '/x_test.npy')
    print("Numpyデータを正常にロードしました。")
except FileNotFoundError:
    print(f"エラー: {data_dir} でデータファイルが見つかりません。")
    print("代替策: カレントディレクトリから .npy ファイルを読み込みます...")
    try:
        x_train = np.load('x_train.npy')
        t_train = np.load('y_train.npy')
        x_test = np.load('x_test.npy')
        work_dir = '.' # 保存先をカレントに
        data_dir = '.'
        print("カレントディレクトリからデータを読み込みました。")
    except FileNotFoundError:
        print("カレントディレクトリにもファイルがありません。処理を中断します。")
        exit()


# --- データの前処理 (Numpy) ---
x_train, x_test = x_train / 255., x_test / 255.
x_train = x_train.reshape(x_train.shape[0], -1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], -1).astype('float32')
t_train_onehot = np.eye(N=10)[t_train.astype("int32").flatten()].astype('float32')

# 検証データの分割 (BASE_SEED=42 で固定)
x_train_split, x_valid_split, t_train_split_onehot, t_valid_split_onehot = train_test_split(
    x_train, t_train_onehot, test_size=0.2, random_state=BASE_SEED
)

# --- PyTorch Dataset/DataLoader (ユーザー定義ベース) ---
class FashionDataset(torch.utils.data.Dataset):
    def __init__(self, x_data, t_data_onehot):
        self.x_data = torch.tensor(x_data, dtype=torch.float)
        self.t_data_onehot = torch.tensor(t_data_onehot, dtype=torch.float)
        self.t_data_label = torch.tensor(np.argmax(t_data_onehot, axis=1), dtype=torch.long)

    def __len__(self):
        return self.x_data.shape[0]

    def __getitem__(self, idx):
        return self.x_data[idx], self.t_data_onehot[idx], self.t_data_label[idx]

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = torch.tensor(x_test, dtype=torch.float)

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return self.x_test[idx]

# DataLoader の設定
batch_size = 128
train_data = FashionDataset(x_train_split, t_train_split_onehot)
valid_data = FashionDataset(x_valid_split, t_valid_split_onehot)
test_data = TestDataset(x_test)

# (DataLoaderは学習ループ内でシード毎に再生成)
print("\n--- データセット準備完了 ---")


# ===================================================================
# 2. 改善策 (安定化、拡張、TTA)
# ===================================================================

# --- 1. 補助関数 (ReLU) ---
def relu(x):
    # ユーザーコードの relu (torch.where) は低速な可能性があるため、clampに変更
    return torch.clamp(x, min=0.0)
    # return torch.where(x > 0, x, torch.zeros_like(x)) # ユーザー版

# --- 改善策 1: 数値的に安定な損失関数 (LogSumExp) ---
def stable_cross_entropy_loss(logits, t_hot):
    """
    ロジット (logits) を入力とし、数値的に安定なクロスエントロピー損失を計算
    """
    # LogSumExpトリック: log(softmax(x)) = x - log(sum(exp(x)))
    log_softmax_manual = logits - torch.logsumexp(logits, dim=1, keepdim=True)
    # NLLLoss (Negative Log Likelihood Loss)
    return -(t_hot * log_softmax_manual).sum(dim=1).mean()

# --- 改善策 2: データ拡張 (Shift, Flip + Cutout) ---
def augment_data_torch_cutout(images_784, h_flip_prob=0.1, shift_pixels=1, cutout_size=8):
    """
    (N, 784) のテンソルを入力とし、Shift, Flip, Cutout を適用
    """
    images_2d = images_784.reshape(-1, 28, 28)
    n_images, h, w = images_2d.shape
    device = images_784.device

    # --- 1. Shift (±1ピクセル) ---
    shifts = torch.randint(-shift_pixels, shift_pixels + 1, (n_images, 2), device=device)
    augmented_images = torch.zeros_like(images_2d) # Shift適用後のテンポラリ

    for i in range(n_images):
        img = images_2d[i]
        shift_y, shift_x = shifts[i, 0].item(), shifts[i, 1].item()
        y_start_dest, y_end_dest = max(0, shift_y), min(h, h + shift_y)
        x_start_dest, x_end_dest = max(0, shift_x), min(w, w + shift_x)
        y_start_src, y_end_src = max(0, -shift_y), min(h, h - shift_y)
        x_start_src, x_end_src = max(0, -shift_x), min(w, w - shift_x)
        if x_start_dest < x_end_dest and y_start_dest < y_end_dest:
            augmented_images[i, y_start_dest:y_end_dest, x_start_dest:x_end_dest] = \
                img[y_start_src:y_end_src, x_start_src:x_end_src]

    # --- 2. Horizontal Flip (10%の確率) ---
    flip_mask = torch.rand(n_images, device=device) < h_flip_prob
    if flip_mask.any():
        augmented_images[flip_mask] = torch.flip(augmented_images[flip_mask], dims=[2])

    # --- 3. Cutout (★ 改善点) ---
    if cutout_size > 0:
        # (N) 個のランダムな座標を生成 (バッチ処理)
        y_starts = torch.randint(0, h - cutout_size + 1, (n_images,), device=device)
        x_starts = torch.randint(0, w - cutout_size + 1, (n_images,), device=device)
        for i in range(n_images):
            # 矩形領域を 0 でマスク
            y1, y2 = y_starts[i], y_starts[i] + cutout_size
            x1, x2 = x_starts[i], x_starts[i] + cutout_size
            augmented_images[i, y1:y2, x1:x2] = 0.0

    return augmented_images.reshape(-1, 784)


# --- 改善策 3: TTA (Test-Time Augmentation) ---
def predict_with_tta(model, x_784):
    """
    TTAを適用し、平均ロジット（logits）を返す
    """
    model.eval() # 念のため評価モード
    # 1. 元画像のロジット
    logits_original = model(x_784)

    # 2. 反転画像のロジット
    x_2d = x_784.reshape(-1, 28, 28)
    x_2d_flipped = torch.flip(x_2d, dims=[2]) # W次元(dim=2)で反転
    x_784_flipped = x_2d_flipped.reshape(-1, 784)
    logits_flipped = model(x_784_flipped)

    # 3. 平均ロジット
    return (logits_original + logits_flipped) / 2.0


# ===================================================================
# 3. カスタムモジュール (ユーザーコードベース)
# ===================================================================

# 3.1. Dense (Linear) レイヤ
class Dense(nn.Module):
    def __init__(self, in_dim, out_dim, is_relu=True):
        super().__init__()
        # ユーザーコードの一様分布初期化を維持
        scale = math.sqrt(2.0 / in_dim) if is_relu else math.sqrt(1.0 / in_dim)
        W_data = (torch.rand(in_dim, out_dim) * 2 - 1) * scale
        self.W = nn.Parameter(W_data.float())
        b_data = torch.zeros(out_dim)
        self.b = nn.Parameter(b_data.float())

    def forward(self, x):
        return torch.matmul(x, self.W) + self.b

# 3.2. BatchNorm1d レイヤ (ユーザーコードのロジックを維持)
class BatchNorm1d(nn.Module):
    def __init__(self, dim, momentum=0.9, eps=1e-8):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(dim).float())
        self.beta = nn.Parameter(torch.zeros(dim).float())
        self.register_buffer('running_mean', torch.zeros(dim).float())
        self.register_buffer('running_var', torch.ones(dim).float()) # 1で初期化
        self.momentum = momentum
        self.eps = eps

    def forward(self, x):
        if self.training:
            if x.shape[0] > 1:
                mu = x.mean(axis=0)
                var = x.var(axis=0, unbiased=False)
            else:
                mu = x.mean(axis=0)
                var = torch.zeros_like(mu)

            # Numpy版の momentum (0.9) をそのまま使用するロジック (ユーザーコード準拠)
            self.running_mean.copy_(self.momentum * self.running_mean.detach() + (1.0 - self.momentum) * mu.detach())
            self.running_var.copy_(self.momentum * self.running_var.detach() + (1.0 - self.momentum) * var.detach())
            x_norm = (x - mu) / torch.sqrt(var + self.eps)
        else:
            x_norm = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)
        return self.gamma * x_norm

# 3.3. Dropout レイヤ
class Dropout(nn.Module):
    def __init__(self, dropout_ratio=0.5):
        super().__init__()
        self.dropout_ratio = dropout_ratio
        self.scale = 1.0 / (1.0 - self.dropout_ratio) if self.dropout_ratio < 1.0 else 1.0

    def forward(self, x):
        if self.training and self.dropout_ratio > 0.0:
            drop_mask = (torch.rand_like(x) > self.dropout_ratio).float() * self.scale
            return x * drop_mask
        else:
            return x

# 3.4. MLPモデル (★ 改善: ロジットを返すように変更)
class MLP(nn.Module):
    def __init__(self, in_dim, hidden_sizes, out_dim, dropout_ratio, bn_momentum):
        super().__init__()

        # ユーザーコードの Sequential + 手動ReLU の構成を維持
        self.hidden_layers = nn.Sequential(
            Dense(in_dim, hidden_sizes[0], is_relu=True),
            BatchNorm1d(hidden_sizes[0], momentum=bn_momentum),
            Dropout(dropout_ratio),

            Dense(hidden_sizes[0], hidden_sizes[1], is_relu=True),
            BatchNorm1d(hidden_sizes[1], momentum=bn_momentum),
            Dropout(dropout_ratio),

            Dense(hidden_sizes[1], hidden_sizes[2], is_relu=True),
            BatchNorm1d(hidden_sizes[2], momentum=bn_momentum),
            Dropout(dropout_ratio)
        )
        self.output_layer = Dense(hidden_sizes[2], out_dim, is_relu=False)

    def forward(self, x):
        # ユーザーコードのフォワードパスを維持
        x = self.hidden_layers[0](x) # Dense 1
        x = self.hidden_layers[1](x) # BN 1
        x = self.hidden_layers[2](x) # Dropout 1
        x = relu(x) # ReLU 1

        x = self.hidden_layers[3](x) # Dense 2
        x = self.hidden_layers[4](x) # BN 2
        x = self.hidden_layers[5](x) # Dropout 2
        x = relu(x) # ReLU 2

        x = self.hidden_layers[6](x) # Dense 3
        x = self.hidden_layers[7](x) # BN 3
        x = self.hidden_layers[8](x) # Dropout 3
        x = relu(x) # ReLU 3

        logits = self.output_layer(x)

        # ★ 改善: softmax を削除し、ロジットをそのまま返す
        # y = softmax(x)
        # return y
        return logits


# ===================================================================
# 4. 学習・評価ループ (改善策適用)
# ===================================================================

def train_one_epoch(model, loader, optimizer, config):
    model.train() # 訓練モード
    total_loss = 0
    total_correct = 0
    total_samples = 0

    # (x, t_hot, t_label) を受け取る
    for x, t_hot, t_label in loader:

        # --- 改善策 2: Cutout適用 ---
        x_aug = augment_data_torch_cutout(
            x,
            h_flip_prob=config['h_flip_prob'],
            shift_pixels=config['shift_pixels'],
            cutout_size=config['cutout_size']
        )
        x_aug, t_hot, t_label = x_aug.to(device), t_hot.to(device), t_label.to(device)

        # 順伝播 (ロジットを取得)
        y_logits = model(x_aug)

        # --- 改善策 1: 安定な損失関数 ---
        loss = stable_cross_entropy_loss(y_logits, t_hot)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        pred = y_logits.argmax(1)
        total_correct += (pred == t_label).sum().item()
        total_samples += x.size(0)

    return total_loss / total_samples, total_correct / total_samples

def validate_one_epoch(model, loader):
    model.eval() # 評価モード
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for x, t_hot, t_label in loader:
            x, t_hot, t_label = x.to(device), t_hot.to(device), t_label.to(device)

            # --- 改善策 3: TTA適用 ---
            y_logits_avg = predict_with_tta(model, x)

            # --- 改善策 1: 安定な損失関数 ---
            loss = stable_cross_entropy_loss(y_logits_avg, t_hot)

            total_loss += loss.item() * x.size(0)
            pred = y_logits_avg.argmax(1)
            total_correct += (pred == t_label).sum().item()
            total_samples += x.size(0)

    return total_loss / total_samples, total_correct / total_samples


# ===================================================================
# 5. メイン実行ブロック (改善策 4: アンサンブル学習)
# ===================================================================

# --- 5.1. ハイパーパラメータ設定 ---
in_dim = 784
out_dim = 10
hidden_sizes = [1024, 512, 256]
bn_momentum = 0.9    # Numpy版のmomentum値 (ユーザーコード準拠)
lr = 0.001           # 学習率 (Adam)
reg_lambda = 1e-5    # L2正則化 (weight_decay)

n_epochs = 500       # 総エポック数 (コサインアニーリングのため)
patience = 60        # 早期終了のPatience (チェック6回分)

# 改善策パラメータ
config = {
    'dropout_ratio': 0.15, # ユーザー設定値 (0.25~0.35で調整可)
    'h_flip_prob': 0.5,   # ユーザー設定値
    'shift_pixels': 1,
    'cutout_size': 6      # ★ Cutout有効化
}

# --- 改善策 4: アンサンブル ---
ensemble_seeds = [42, 123, 555] # 3つの異なるシード
all_test_logits = [] # 各モデルのテストロジットを保存するリスト

print(f"\n--- アンサンブル学習開始 (Models: {len(ensemble_seeds)}) ---")
print(f"--- 改善策: StableLoss, Cutout(size={config['cutout_size']}), TTA ---")

overall_start_time = time.time()

# DataLoader (アンサンブルループの外で定義)
# DataLoaderのワーカ間でシードが異なるようにする設定
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(BASE_SEED) # DataLoaderのshuffle用シード

dataloader_train = torch.utils.data.DataLoader(
    train_data, batch_size=batch_size, shuffle=True,
    worker_init_fn=seed_worker, generator=g
)
dataloader_valid = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=False)
dataloader_test = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


for i, seed in enumerate(ensemble_seeds):
    print(f"\n--- モデル {i+1}/{len(ensemble_seeds)} (Seed: {seed}) の学習開始 ---")
    model_start_time = time.time()

    # シード固定 (モデル初期化、Dropout, Augmentationのため)
    set_seed(seed)

    mlp = MLP(
        in_dim,
        hidden_sizes,
        out_dim,
        config['dropout_ratio'],
        bn_momentum
    ).to(device)

    optimizer = optim.Adam(mlp.parameters(), lr=lr, weight_decay=reg_lambda)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

    best_valid_accuracy = 0.0
    patience_counter = 0
    best_model_state = None

    for epoch in range(n_epochs):
        train_loss, train_acc = train_one_epoch(mlp, dataloader_train, optimizer, config)
        valid_loss, valid_acc = validate_one_epoch(mlp, dataloader_valid)
        scheduler.step()

        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'EPOCH: {epoch+1:3d}, Train [Acc: {train_acc:.4f}], '
                  f'Valid [Acc (TTA): {valid_acc:.4f}], '
                  f'LR: {optimizer.param_groups[0]["lr"]:.6f}')

        if valid_acc > best_valid_accuracy:
            best_valid_accuracy = valid_acc
            patience_counter = 0
            best_model_state = copy.deepcopy(mlp.state_dict())
            if (epoch + 1) % 10 == 0 or epoch == 0:
                 print(f"    -> Best Valid Accuracy Updated: {best_valid_accuracy:.4f}")
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"\n--- Early stopping triggered after {epoch+1} epochs. ---")
            break

    model_end_time = time.time()
    print(f"--- モデル {i+1} 学習完了 (Time: {model_end_time - model_start_time:.1f} sec) ---")
    print(f"    Best Valid Accuracy (TTA): {best_valid_accuracy:.4f}")

    # --- 5.2. テストデータでの予測 (TTA使用) ---
    if best_model_state:
        mlp.load_state_dict(best_model_state)

    mlp.eval()
    model_test_logits = []
    with torch.no_grad():
        for x_test_batch in dataloader_test:
            x_test_batch = x_test_batch.to(device)
            logits_avg = predict_with_tta(mlp, x_test_batch)
            model_test_logits.append(logits_avg.cpu())

    all_test_logits.append(torch.cat(model_test_logits, dim=0))


# --- 5.3. アンサンブルによる最終予測 ---
print("\n--- 全モデルの学習完了 ---")
print("--- アンサンブルによる最終予測 (TTA + Ensemble) ---")

# (NumModels, N_Test, NumClasses) のテンソルにスタック
stacked_logits = torch.stack(all_test_logits)

# NumModels 次元で平均 (N_Test, NumClasses)
avg_logits = torch.mean(stacked_logits, dim=0)

# 最終予測
predictions = torch.argmax(avg_logits, dim=1).numpy()
submission = pd.Series(predictions, name='label')

# 保存先パスの作成 (ユーザーコードのパスを尊重)
submission_filename = os.path.join(work_dir, 'Lecture04', 'submission_pred_04_optimized_final.csv')
os.makedirs(os.path.dirname(submission_filename), exist_ok=True)

submission.to_csv(submission_filename, header=True, index_label='id')

overall_end_time = time.time()
print(f"\n--- 処理完了 (Total Time: {overall_end_time - overall_start_time:.1f} sec) ---")
print(f"最適化された予測ファイルが保存されました: {submission_filename}")
print("目標スコア 0.95 の達成を期待します。")


Mounted at /content/drive
データディレクトリ: /content/drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture04/data
デバイス: cuda
Numpyデータを正常にロードしました。

--- データセット準備完了 ---

--- アンサンブル学習開始 (Models: 3) ---
--- 改善策: StableLoss, Cutout(size=6), TTA ---

--- モデル 1/3 (Seed: 42) の学習開始 ---
EPOCH:   1, Train [Acc: 0.7618], Valid [Acc (TTA): 0.8273], LR: 0.001000
    -> Best Valid Accuracy Updated: 0.8273
EPOCH:  10, Train [Acc: 0.8472], Valid [Acc (TTA): 0.8732], LR: 0.000999
    -> Best Valid Accuracy Updated: 0.8732
EPOCH:  20, Train [Acc: 0.8596], Valid [Acc (TTA): 0.8877], LR: 0.000996
EPOCH:  30, Train [Acc: 0.8683], Valid [Acc (TTA): 0.8888], LR: 0.000991
EPOCH:  40, Train [Acc: 0.8730], Valid [Acc (TTA): 0.8898], LR: 0.000984
EPOCH:  50, Train [Acc: 0.8755], Valid [Acc (TTA): 0.8984], LR: 0.000976
EPOCH:  60, Train [Acc: 0.8781], Valid [Acc (TTA): 0.8952], LR: 0.000965
EPOCH:  70, Train [Acc: 0.8818], Valid [Acc (TTA): 0.8986], LR: 0.000952
EPOCH:  80, Train [Acc: 0.8842], Valid [Acc (TTA): 0.9029

In [3]:
# ===================================================================
# 0. 環境設定とAPI制限 (★「コードB」の安全なAPI制限を導入)
# ===================================================================

import os
import inspect
import copy
import math
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- Google Driveマウントと作業ディレクトリ設定 ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    # ★★★ ご自身の環境に合わせて修正してください ★★★
    work_dir = '/content/drive/MyDrive/Colab Notebooks/DLBasics2025_colab'
    if not os.path.exists(work_dir):
        print(f"警告: ディレクトリ '{work_dir}' が存在しません。'.' を使用します。")
        work_dir = '.'
except ImportError:
    print("Google Colab 環境ではありません。カレントディレクトリ '.' を使用します。")
    work_dir = '.'
print(f"作業ディレクトリ: {work_dir}")


# ★★★ 改善点 1: 「コードB」で使われていた安全なAPI制限に変更 ★★★
# optim.Adamが依存する内部モジュール(functional等)を削除しません。
nn_except = ["Module", "Parameter", "Sequential", "parameter", "init", "CrossEntropyLoss", "ModuleList", "ModuleDict"] # Added ModuleDict

for m in inspect.getmembers(nn):
    attr_name = m[0]
    attr_value = m[1]

    # 除外リストまたはプライベート属性はスキップ
    if attr_name in nn_except or attr_name.startswith("__"):
        continue

    # ★重要: モジュール型の属性は削除しない (functional, attention等)
    if inspect.ismodule(attr_value):
        continue

    # それ以外のクラス・関数(nn.Linear, nn.ReLUなど)を削除
    try:
        delattr(nn, attr_name)
    except (AttributeError, TypeError):
        pass

print(f"--- API制限適用完了 (許可: {nn_except} + サブモジュール) ---")
# ★★★ 改善完了 ★★★


# --- シード固定関数 ---
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # アンサンブルのためにallも設定
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# --- デバイス設定 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")


# ===================================================================
# 1. データ準備 (★改善点 2: Cutoutのバグを今度こそ修正)
# ===================================================================

def augment_data_torch_cutout(images_flat, h_flip_prob=0.5, shift_pixels=2, cutout_size=8):
    """
    torchvisionに依存しない、テンソル演算のみによるデータ拡張。
    """
    images_2d = images_flat.reshape(-1, 1, 28, 28)
    n_images, c, h, w = images_2d.shape

    # --- 1. 水平フリップ (p=0.5) ---
    flip_mask = torch.rand(n_images, device=images_flat.device) < h_flip_prob
    if flip_mask.any():
        images_2d[flip_mask] = torch.flip(images_2d[flip_mask], dims=[3])

    # --- 2. ランダムシフト (±2ピクセル) ---
    shifts = torch.randint(-shift_pixels, shift_pixels + 1, (n_images, 2), device=images_flat.device)
    # nn.functional.pad は安全なAPI制限により削除されていないため使用可能
    padded_images = torch.nn.functional.pad(images_2d, (shift_pixels, shift_pixels, shift_pixels, shift_pixels), mode='constant', value=0.0)
    augmented_shifted = torch.empty_like(images_2d)

    for i in range(n_images):
        shift_y, shift_x = shifts[i, 0].item(), shifts[i, 1].item()
        y_start = shift_pixels + shift_y
        x_start = shift_pixels + shift_x
        augmented_shifted[i] = padded_images[i, :, y_start:y_start+h, x_start:x_start+w]

    images_2d = augmented_shifted

    # --- 3. カットアウト (Cutout) ---
    cutout_half = cutout_size // 2
    center_y = torch.randint(0, h, (n_images,), device=images_flat.device)
    center_x = torch.randint(0, w, (n_images,), device=images_flat.device)

    for i in range(n_images):
        y1 = torch.clamp(center_y[i] - cutout_half, 0, h).item()
        y2 = torch.clamp(center_y[i] + cutout_half, 0, h).item()

        # ★★★ 改善点 2: バグ修正 ★★★
        # x1 の計算が両方 '+' になっていたのを '-' に修正
        # x1 = torch.clamp(center_x[i] + cutout_half, 0, w).item() # <- バグ
        x1 = torch.clamp(center_x[i] - cutout_half, 0, w).item() # <- 修正！
        x2 = torch.clamp(center_x[i] + cutout_half, 0, w).item()

        if y1 < y2 and x1 < x2:
            images_2d[i, :, y1:y2, x1:x2] = 0.0
    # ★★★ 改善完了 ★★★

    return images_2d.reshape(-1, 784)

# --- データセットクラス (変更なし) ---
class FashionDataset(Dataset):
    def __init__(self, x_data, t_label):
        # ★改善: 0-1に正規化 (augment_data_torch_cutoutは0-1を前提)
        self.x_data = x_data.reshape(-1, 784).astype(np.float32) / 255.0
        self.t_label = t_label
        self.t_hot = np.eye(10)[t_label].astype(np.float32)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        img_tensor = torch.tensor(self.x_data[idx], dtype=torch.float32)
        label = torch.tensor(self.t_label[idx], dtype=torch.long)
        hot_label = torch.tensor(self.t_hot[idx], dtype=torch.float32)
        return img_tensor, hot_label, label

class TestDataset(Dataset):
    def __init__(self, x_data):
        self.x_data = x_data.reshape(-1, 784).astype(np.float32) / 255.0

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        img_tensor = torch.tensor(self.x_data[idx], dtype=torch.float32)
        return img_tensor

# --- データのロードと分割 ---
data_path = os.path.join(work_dir, 'Lecture04', 'data')
x_train_all = np.load(os.path.join(data_path, 'x_train.npy'))
t_train_all = np.load(os.path.join(data_path, 'y_train.npy'))
x_test = np.load(os.path.join(data_path, 'x_test.npy'))

x_train_split, x_valid_split, t_train_split_label, t_valid_split_label = train_test_split(
    x_train_all, t_train_all, test_size=0.2, random_state=42
)
print(f"訓練データ: {x_train_split.shape}, 検証データ: {x_valid_split.shape}")

# --- データセットインスタンスの作成 ---
train_data = FashionDataset(x_train_split, t_train_split_label)
valid_data = FashionDataset(x_valid_split, t_valid_split_label)
test_data = TestDataset(x_test)


# --- データローダーの作成 ---
batch_size = 128
num_workers = 2

dataloader_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
dataloader_valid = DataLoader(valid_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
dataloader_test = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

print("--- データローダー準備完了 (torchvision非依存) ---")


# ===================================================================
# 2. PyTorchによるニューラルネットワークのクラス化 (変更なし)
# ===================================================================

def relu(x):
    return torch.where(x > 0, x, torch.zeros_like(x))

class Dense(nn.Module):
    def __init__(self, in_dim, out_dim, is_relu=True):
        super().__init__()
        self.W = nn.Parameter(torch.empty(in_dim, out_dim, dtype=torch.float32))
        self.b = nn.Parameter(torch.empty(out_dim, dtype=torch.float32))
        self.initialize_parameters(is_relu)

    def initialize_parameters(self, is_relu):
        if is_relu:
            nn.init.kaiming_normal_(self.W, mode='fan_in', nonlinearity='relu')
        else:
            nn.init.xavier_normal_(self.W)
        nn.init.zeros_(self.b)

    def forward(self, x):
        return torch.matmul(x, self.W) + self.b

class BatchNorm1d(nn.Module):
    def __init__(self, dim, momentum=0.9, eps=1e-8):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(dim).float())
        self.beta = nn.Parameter(torch.zeros(dim).float())
        self.register_buffer('running_mean', torch.zeros(dim).float())
        self.register_buffer('running_var', torch.ones(dim).float())
        self.momentum = momentum
        self.eps = eps

    def forward(self, x):
        if self.training:
            if x.shape[0] > 1:
                mu = x.mean(axis=0)
                var = x.var(axis=0, unbiased=False)
            else:
                mu = x.mean(axis=0)
                var = torch.zeros_like(mu)
            self.running_mean = self.momentum * self.running_mean.detach() + (1.0 - self.momentum) * mu.detach()
            self.running_var = self.momentum * self.running_var.detach() + (1.0 - self.momentum) * var.detach()
            x_norm = (x - mu) / torch.sqrt(var + self.eps)
        else:
            x_norm = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)
        return self.gamma * x_norm + self.beta

class Dropout(nn.Module):
    def __init__(self, dropout_ratio=0.5):
        super().__init__()
        self.dropout_ratio = dropout_ratio

    def forward(self, x):
        if self.training and self.dropout_ratio > 0.0:
            drop_mask = (torch.rand_like(x) > self.dropout_ratio).float() / (1.0 - self.dropout_ratio)
            return x * drop_mask
        else:
            return x

class MLP(nn.Module):
    def __init__(self, in_dim, hidden_sizes, out_dim, dropout_ratio, bn_momentum):
        super().__init__()
        self.fc1 = Dense(in_dim, hidden_sizes[0], is_relu=True)
        self.bn1 = BatchNorm1d(hidden_sizes[0], momentum=bn_momentum)
        self.drop1 = Dropout(dropout_ratio)
        self.fc2 = Dense(hidden_sizes[0], hidden_sizes[1], is_relu=True)
        self.bn2 = BatchNorm1d(hidden_sizes[1], momentum=bn_momentum)
        self.drop2 = Dropout(dropout_ratio)
        self.fc3 = Dense(hidden_sizes[1], hidden_sizes[2], is_relu=True)
        self.bn3 = BatchNorm1d(hidden_sizes[2], momentum=bn_momentum)
        self.drop3 = Dropout(dropout_ratio)
        self.output_layer = Dense(hidden_sizes[2], out_dim, is_relu=False)

    def forward(self, x):
        x = relu(self.drop1(self.bn1(self.fc1(x))))
        x = relu(self.drop2(self.bn2(self.fc2(x))))
        x = relu(self.drop3(self.bn3(self.fc3(x))))
        x = self.output_layer(x)
        return x


# ===================================================================
# 3. アンサンブル学習ループ (★改善点 3: dropout_ratio=0.3)
# ===================================================================

in_dim = 784
out_dim = 10
hidden_sizes = [1024, 512, 256]
bn_momentum = 0.9
lr = 0.001
reg_lambda = 1e-5

# ★★★ 改善点 3: 正則化のバランス調整 ★★★
# 学習不足(Underfitting)を解消するため、Dropoutを 0.4 から 0.3 に戻します。
dropout_ratio = 0.3 # 0.4
# ★★★ 改善完了 ★★★

n_epochs = 500
patience = 60

ensemble_seeds = [42, 123, 555]
all_best_model_states = []
all_best_valid_accs = []

criterion = nn.CrossEntropyLoss()

print(f"--- アンサンブル学習開始 (Models: {len(ensemble_seeds)}, Epochs: {n_epochs}, Dropout: {dropout_ratio}) ---")
print("--- ★データ拡張: 手動Cutout + Shift + Flip (torchvision非依存) ---")

for i, seed in enumerate(ensemble_seeds):
    print(f"\n--- モデル {i+1}/{len(ensemble_seeds)} (Seed: {seed}) の学習開始 ---")

    set_seed(seed)

    mlp = MLP(in_dim, hidden_sizes, out_dim, dropout_ratio, bn_momentum).to(device)

    # ★★★ エラーが発生していた行 ★★★
    # 安全なAPI制限に変更したため、optim.Adam は
    # nn.functional や nn.ModuleDict を見つけることができ、正常に初期化されます。
    optimizer = optim.Adam(mlp.parameters(), lr=lr, weight_decay=reg_lambda)

    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

    best_valid_accuracy = 0.0
    patience_counter = 0
    best_model_state = None

    for epoch in range(n_epochs):
        losses_train = []
        train_num = 0
        train_true_num = 0

        mlp.train()
        for x, t_hot, t_label in dataloader_train:
            x, t_label = x.to(device), t_label.to(device)

            # --- ★手動データ拡張 (バグ修正済みCutout) ---
            x_aug = augment_data_torch_cutout(x,
                                                h_flip_prob=0.5,
                                                shift_pixels=2,
                                                cutout_size=8)

            y_logits = mlp(x_aug)
            loss = criterion(y_logits, t_label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses_train.append(loss.tolist())
            pred = y_logits.argmax(1)
            acc = (pred == t_label).float()
            train_num += acc.size(0)
            train_true_num += acc.sum().item()

        # --- 検証 (★手動TTA: 水平フリップ) ---
        mlp.eval()
        losses_valid = []
        valid_num = 0
        valid_true_num = 0
        with torch.no_grad():
            for x, t_hot, t_label in dataloader_valid:
                x, t_label = x.to(device), t_label.to(device)

                y_logits_original = mlp(x)

                x_reshaped = x.reshape(-1, 1, 28, 28)
                x_flipped = torch.flip(x_reshaped, dims=[3])
                x_flipped_flat = x_flipped.reshape(-1, 784)
                y_logits_flipped = mlp.forward(x_flipped_flat)

                y_logits_avg = (y_logits_original + y_logits_flipped) / 2.0

                loss = criterion(y_logits_avg, t_label)
                losses_valid.append(loss.tolist())

                pred = y_logits_avg.argmax(1)
                acc = (pred == t_label).float()
                valid_num += acc.size(0)
                valid_true_num += acc.sum().item()

        scheduler.step()

        current_train_accuracy = train_true_num / train_num
        current_valid_accuracy = valid_true_num / valid_num
        current_lr = optimizer.param_groups[0]['lr']

        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'EPOCH: {epoch+1:3d}, Train [Loss: {np.mean(losses_train):.4f}, Acc: {current_train_accuracy:.4f}], '
                  f'Valid [Loss: {np.mean(losses_valid):.4f}, Acc(TTA): {current_valid_accuracy:.4f}], '
                  f'LR: {current_lr:.6f}')

        if current_valid_accuracy > best_valid_accuracy:
            best_valid_accuracy = current_valid_accuracy
            patience_counter = 0
            best_model_state = copy.deepcopy(mlp.state_dict())
            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f"    -> Best TTA Valid Accuracy Updated: {best_valid_accuracy:.4f}")
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"\n--- Early stopping triggered after {epoch+1} epochs. ---")
            break

    all_best_model_states.append(best_model_state)
    all_best_valid_accs.append(best_valid_accuracy)
    print(f"--- モデル {i+1} 完了 (Best Valid Acc: {best_valid_accuracy:.4f}) ---")


# ===================================================================
# 4. 最終予測 (アンサンブルと手動TTA) (変更なし)
# ===================================================================
print("\n--- 全モデルの学習完了 ---")
print(f"各モデルのベスト検証精度 (TTA): {[round(acc, 4) for acc in all_best_valid_accs]}")
print(f"平均検証精度 (TTA): {np.mean(all_best_valid_accs):.4f}")
print("\n--- アンサンブルとTTAによる最終予測の生成 ---")

all_test_logits = []

mlp = MLP(in_dim, hidden_sizes, out_dim, dropout_ratio, bn_momentum).to(device)
mlp.eval()

for model_state in all_best_model_states:
    if model_state is None:
        print("警告: 訓練されなかったモデル（stateがNone）をスキップします。")
        continue

    mlp.load_state_dict(model_state)

    model_test_logits = []

    with torch.no_grad():
        for x in dataloader_test:
            x = x.to(device)

            y_logits_original = mlp.forward(x)

            x_reshaped = x.reshape(-1, 1, 28, 28)
            x_flipped = torch.flip(x_reshaped, dims=[3])
            x_flipped_flat = x_flipped.reshape(-1, 784)
            y_logits_flipped = mlp.forward(x_flipped_flat)

            y_logits_avg = (y_logits_original + y_logits_flipped) / 2.0

            model_test_logits.append(y_logits_avg.cpu())

    all_test_logits.append(torch.cat(model_test_logits, dim=0))

if not all_test_logits:
    print("エラー: 有効なモデルが1つも訓練されませんでした。")
else:
    stacked_logits = torch.stack(all_test_logits, dim=0)
    mean_logits = torch.mean(stacked_logits, dim=0)
    final_pred = mean_logits.argmax(1).tolist()

    # --- 提出ファイルの保存 ---
    submission = pd.Series(final_pred, name='label')
    submission_filename = os.path.join(work_dir, 'Lecture04', 'submission_pred_11_final_fixed.csv')
    os.makedirs(os.path.dirname(submission_filename), exist_ok=True)
    submission.to_csv(submission_filename, header=True, index_label='id')
    print(f"\n--- 予測完了。提出ファイル '{submission_filename}' を保存しました。 ---")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
作業ディレクトリ: /content/drive/MyDrive/Colab Notebooks/DLBasics2025_colab
--- API制限適用完了 (許可: ['Module', 'Parameter', 'Sequential', 'parameter', 'init', 'CrossEntropyLoss', 'ModuleList', 'ModuleDict'] + サブモジュール) ---
使用デバイス: cuda
訓練データ: (48000, 28, 28), 検証データ: (12000, 28, 28)
--- データローダー準備完了 (torchvision非依存) ---
--- アンサンブル学習開始 (Models: 3, Epochs: 500, Dropout: 0.3) ---
--- ★データ拡張: 手動Cutout + Shift + Flip (torchvision非依存) ---

--- モデル 1/3 (Seed: 42) の学習開始 ---
EPOCH:   1, Train [Loss: 0.9033, Acc: 0.6624], Valid [Loss: 0.5901, Acc(TTA): 0.7642], LR: 0.001000
    -> Best TTA Valid Accuracy Updated: 0.7642
EPOCH:  10, Train [Loss: 0.5509, Acc: 0.7960], Valid [Loss: 0.4017, Acc(TTA): 0.8488], LR: 0.000999
    -> Best TTA Valid Accuracy Updated: 0.8488
EPOCH:  20, Train [Loss: 0.4875, Acc: 0.8183], Valid [Loss: 0.3569, Acc(TTA): 0.8650], LR: 0.000996
    -> Best TTA Valid 

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a936b79fec0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1628, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
    

KeyboardInterrupt: 

claude

In [5]:
# ================== セットアップ（最初に1回だけ実行）==================
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import inspect

# 使用できるAPIを制限（改良版: サブモジュールは保護）
nn_except = ["Module", "Parameter", "Sequential", "parameter", "ModuleList"] # Added ModuleList
# サブモジュール（削除してはいけない）
nn_submodules = ["attention", "functional", "init", "modules", "utils", "common_types", "grad"]

for m in inspect.getmembers(nn):
    attr_name = m[0]
    # 除外リスト、プライベート属性、サブモジュールはスキップ
    if attr_name in nn_except or attr_name[0:2] == "__" or attr_name in nn_submodules:
        continue
    # モジュール型もスキップ
    if inspect.ismodule(m[1]):
        continue
    # それ以外を削除
    try:
        delattr(nn, attr_name)
    except (AttributeError, TypeError):
        pass  # 削除できない場合はスキップ

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 学習データ
x_train = np.load(work_dir + '/Lecture04/data/x_train.npy')
t_train = np.load(work_dir + '/Lecture04/data/y_train.npy')

# テストデータ
x_test = np.load(work_dir + '/Lecture04/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)
print(f"Train+Val samples: {len(trainval_data)}, Test samples: {len(test_data)}")


# ================== カスタムレイヤの実装 ==================

def relu(x):
    """ReLU活性化関数"""
    return torch.where(x > 0, x, torch.zeros_like(x))


def softmax(x):
    """Softmax関数"""
    exp_x = torch.exp(x - torch.max(x, dim=1, keepdim=True)[0])
    return exp_x / torch.sum(exp_x, dim=1, keepdim=True)


class CustomDense(nn.Module):
    """He初期化を使ったカスタム全結合層"""
    def __init__(self, in_features, out_features):
        super(CustomDense, self).__init__()
        # He初期化（Kaiming初期化）
        std = torch.sqrt(torch.tensor(2.0 / in_features))
        self.W = nn.Parameter(torch.randn(in_features, out_features) * std)
        self.b = nn.Parameter(torch.zeros(out_features))

    def forward(self, x):
        return torch.matmul(x, self.W) + self.b


class CustomBatchNorm1d(nn.Module):
    """カスタムBatch Normalizationレイヤ"""
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(CustomBatchNorm1d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum

        # 学習可能なパラメータ
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))

        # 推論時用の統計量（学習されない）
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        if self.training:
            # 訓練時: バッチの統計量を使用
            batch_mean = torch.mean(x, dim=0)
            batch_var = torch.var(x, dim=0, unbiased=False)

            # 移動平均を更新
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var

            # 正規化
            x_normalized = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
        else:
            # 推論時: 移動平均を使用
            x_normalized = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)

        # スケールとシフト
        return self.gamma * x_normalized + self.beta


class CustomDropout(nn.Module):
    """カスタムDropoutレイヤ"""
    def __init__(self, p=0.5):
        super(CustomDropout, self).__init__()
        self.p = p

    def forward(self, x):
        if self.training:
            # 訓練時: ドロップアウトを適用
            mask = (torch.rand_like(x) > self.p).float()
            return x * mask / (1 - self.p)  # スケーリング
        else:
            # 推論時: 何もしない
            return x


class CustomReLU(nn.Module):
    """カスタムReLUレイヤ"""
    def forward(self, x):
        return relu(x)


# ================== データ拡張関数（テンソルスライスのみ）==================

def augment_data(x, shift_range=2):
    """
    データ拡張: ランダム左右反転とランダムシフト
    nn.functionalを使わず、テンソルのスライス操作のみで実装
    x: shape (batch_size, 784)
    """
    batch_size = x.shape[0]
    x_images = x.view(batch_size, 28, 28)
    x_augmented = torch.zeros_like(x_images)

    for i in range(batch_size):
        img = x_images[i]

        # ランダム左右反転（50%の確率）
        if torch.rand(1).item() > 0.5:
            img = torch.flip(img, dims=[1])

        # ランダムシフト（テンソルスライスで手動実装）
        if shift_range > 0:
            shift_h = torch.randint(-shift_range, shift_range + 1, (1,)).item()
            shift_w = torch.randint(-shift_range, shift_range + 1, (1,)).item()

            # シフト後の画像を作成（ゼロパディング）
            shifted_img = torch.zeros_like(img)

            # コピー元の範囲を計算
            src_h_start = max(0, -shift_h)
            src_h_end = min(28, 28 - shift_h)
            src_w_start = max(0, -shift_w)
            src_w_end = min(28, 28 - shift_w)

            # コピー先の範囲を計算
            dst_h_start = max(0, shift_h)
            dst_h_end = dst_h_start + (src_h_end - src_h_start)
            dst_w_start = max(0, shift_w)
            dst_w_end = dst_w_start + (src_w_end - src_w_start)

            # スライスでコピー
            shifted_img[dst_h_start:dst_h_end, dst_w_start:dst_w_end] = \
                img[src_h_start:src_h_end, src_w_start:src_w_end]

            img = shifted_img

        x_augmented[i] = img

    return x_augmented.view(batch_size, 784)


# ================== MLPモデルの構築 ==================

class MLP(nn.Module):
    """深層MLP"""
    def __init__(self, in_dim, hid_dims, out_dim, dropout_p=0.3):
        super(MLP, self).__init__()

        layers = []
        prev_dim = in_dim

        # 隠れ層
        for hid_dim in hid_dims:
            layers.append(CustomDense(prev_dim, hid_dim))
            layers.append(CustomBatchNorm1d(hid_dim))
            layers.append(CustomReLU())
            layers.append(CustomDropout(dropout_p))
            prev_dim = hid_dim

        # 出力層
        layers.append(CustomDense(prev_dim, out_dim))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


# ================== ハイパーパラメータ設定 ==================

batch_size = 128
val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# モデルのパラメータ
in_dim = 784
hid_dims = [1024, 512, 256, 128]  # 4層の隠れ層
out_dim = 10
dropout_p = 0.4

# 学習パラメータ
lr = 0.001
weight_decay = 1e-4
n_epochs = 400

# モデルとオプティマイザの初期化
mlp = MLP(in_dim, hid_dims, out_dim, dropout_p).to(device)
optimizer = optim.Adam(mlp.parameters(), lr=lr, weight_decay=weight_decay)

# コサインアニーリング学習率スケジューラ
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-6)

print(f"\nModel architecture:")
print(f"Hidden layers: {hid_dims}")
print(f"Dropout: {dropout_p}")
print(f"Learning rate: {lr}")
print(f"Weight decay: {weight_decay}")
print(f"Batch size: {batch_size}")
print(f"Epochs: {n_epochs}\n")


# ================== 訓練ループ ==================

best_valid_acc = 0.0
best_model_state = None

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()
    for x, t in dataloader_train:
        x, t = x.to(device), t.to(device)

        # データ拡張（テンソルスライスのみ使用）
        x = augment_data(x, shift_range=2)

        optimizer.zero_grad()

        # 順伝播
        y = mlp(x)

        # 損失計算（クロスエントロピー）
        log_softmax = torch.log(softmax(y) + 1e-7)
        loss = -torch.mean(log_softmax[torch.arange(t.size(0), device=device), t])

        # 逆伝播
        loss.backward()
        optimizer.step()

        losses_train.append(loss.item())

        # 精度計算
        pred = y.argmax(1)
        acc = torch.where(t - pred == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size(0)
        train_true_num += acc.sum().item()

    mlp.eval()
    with torch.no_grad():
        for x, t in dataloader_valid:
            x, t = x.to(device), t.to(device)

            # 順伝播
            y = mlp(x)

            # 損失計算
            log_softmax = torch.log(softmax(y) + 1e-7)
            loss = -torch.mean(log_softmax[torch.arange(t.size(0), device=device), t])

            losses_valid.append(loss.item())

            # 精度計算
            pred = y.argmax(1)
            acc = torch.where(t - pred == 0, torch.ones_like(t), torch.zeros_like(t))
            valid_num += acc.size(0)
            valid_true_num += acc.sum().item()

    # 学習率の更新
    scheduler.step()

    train_acc = train_true_num / train_num
    valid_acc = valid_true_num / valid_num

    # ベストモデルの保存
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model_state = mlp.state_dict()

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}], LR: {:.6f}'.format(
        epoch + 1,
        np.mean(losses_train),
        train_acc,
        np.mean(losses_valid),
        valid_acc,
        optimizer.param_groups[0]['lr']
    ))

# ベストモデルをロード
mlp.load_state_dict(best_model_state)
print(f'\n{"="*60}')
print(f'Best Validation Accuracy: {best_valid_acc:.4f} ({best_valid_acc*100:.2f}%)')
print(f'{"="*60}\n')


# ================== テストデータの予測 ==================

mlp.eval()
t_pred = []

with torch.no_grad():
    for x in dataloader_test:
        x = x.to(device)

        # 順伝播
        y = mlp(x)

        # モデルの出力を予測値のスカラーに変換
        pred = y.argmax(1).tolist()
        t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv(work_dir + '/Lecture04/submission_pred_04.csv', header=True, index_label='id')

print(f'Predictions saved to submission_pred_04.csv')
print(f'Total test samples: {len(t_pred)}')

Using device: cuda
Train+Val samples: 60000, Test samples: 10000

Model architecture:
Hidden layers: [1024, 512, 256, 128]
Dropout: 0.4
Learning rate: 0.001
Weight decay: 0.0001
Batch size: 128
Epochs: 400

EPOCH: 1, Train [Loss: 0.990, Accuracy: 0.641], Valid [Loss: 0.578, Accuracy: 0.778], LR: 0.001000
EPOCH: 2, Train [Loss: 0.711, Accuracy: 0.737], Valid [Loss: 0.522, Accuracy: 0.803], LR: 0.001000
EPOCH: 3, Train [Loss: 0.646, Accuracy: 0.764], Valid [Loss: 0.474, Accuracy: 0.820], LR: 0.001000
EPOCH: 4, Train [Loss: 0.613, Accuracy: 0.778], Valid [Loss: 0.471, Accuracy: 0.824], LR: 0.001000
EPOCH: 5, Train [Loss: 0.589, Accuracy: 0.785], Valid [Loss: 0.449, Accuracy: 0.830], LR: 0.001000
EPOCH: 6, Train [Loss: 0.569, Accuracy: 0.794], Valid [Loss: 0.439, Accuracy: 0.834], LR: 0.000999
EPOCH: 7, Train [Loss: 0.559, Accuracy: 0.795], Valid [Loss: 0.430, Accuracy: 0.837], LR: 0.000999
EPOCH: 8, Train [Loss: 0.546, Accuracy: 0.802], Valid [Loss: 0.430, Accuracy: 0.834], LR: 0.000999
E

Exception ignored in: <function _releaseLock at 0x7b54b2da56c0>
Traceback (most recent call last):
  File "/usr/lib/python3.12/logging/__init__.py", line 243, in _releaseLock
    def _releaseLock():
    
KeyboardInterrupt: 


KeyboardInterrupt: 